In [7]:
import requests
from bs4 import BeautifulSoup


In [8]:
url = "https://www.senate.gov/legislative/LIS/roll_call_lists/vote_menu_114_1.htm"
headers = {
    "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/120.0 Safari/537.36"
}

response = requests.get(url, headers=headers)
print("Status Code:", response.status_code)


Status Code: 200


In [9]:
soup = BeautifulSoup(response.text, "html.parser")
print("✅ HTML parsed successfully!")


✅ HTML parsed successfully!


In [10]:
# The issue columns usually have class "roll_call" or "issue"; let's extract all links in the table
issues = []
table = soup.find("table")
if table:
    for a in table.find_all("a"):
        issue_text = a.get_text(strip=True)
        if issue_text:  # avoid empty
            issues.append(issue_text)

# Remove duplicates if needed
issues = list(dict.fromkeys(issues))

print("List of issues voted in congress:", issues[:10])  # limiting first 10 for display


List of issues voted in congress: ['339\xa0(65-33)', 'H.R. 2029', '338\xa0(73-25)', '337\xa0(31-67)', '336\xa0(72-26)', '335\xa0(45-34)', 'PN64', '334\xa0(85-12)', 'S. 1177', '333\xa0(84-12)']


In [11]:
# Example: get vote info for the first issue
first_issue_url = "https://www.senate.gov" + soup.find("a", string=issues[0].capitalize()).get("href")
response_issue = requests.get(first_issue_url, headers=headers)
soup_issue = BeautifulSoup(response_issue.text, "html.parser")

# Find all rows in the vote table
vote_rows = soup_issue.find_all("tr")[1:]  # skip header row

for row in vote_rows[:3]:  # show first 3 rows as sample
    cols = row.find_all("td")
    if len(cols) >= 6:
        print("Vote (Tally) ResultQuestion:", "".join([c.get_text(strip=True) for c in cols]))
